In [1]:
import cv2
import time
import src.viztools as vv
from __future__ import print_function

In [ ]:
reload(vv)

In [2]:
cap = cv2.VideoCapture(0)
target_fps = 20.0

key_list = [False for i in range(256)]
while(cap.isOpened()):

    time_pre = time.time()
    
    ret, frame = cap.read()
    
    key = cv2.waitKey(1) & 0xFF
    if key == 27:
        break
    
    key_list[key] = True
    
    vv.test(key_list)
    
    cv2.imshow('frame', frame)
    
    # calculate, limit and output fps
    time_post = time.time()
    time_tot = time_post - time_pre
    if time_tot < 1/target_fps:
        time.sleep(1/target_fps - time_tot)
    
cap.release()
cv2.destroyWindow('frame')

118
100


In [ ]:
# general parameters
USE_CAM = 0
DISP_FULL_SCREEN = 0
DISP_BW = False
LOOP = True    # loop through videos

# parameters shared across effects
INCREMENT = False
DECREMENT = False
USE_ALL_CHAN = True
USE_R_CHAN = False
USE_G_CHAN = False
USE_B_CHAN = False

# keys shared across effects
KEY_PRESS_ZERO = False
KEY_PRESS_ONE = False
KEY_PRESS_TWO = False
KEY_PRESS_THREE = False
KEY_PRESS_MINUS = False # -
KEY_PRESS_PLUS = False  # =
KEY_PRESS_RESET = False # /

"""BORDERS"""
# keyboard shortcut - z
BORDER_STYLE = 0
HEIGHT_MULT = 0.5
WIDTH_MULT = 0.5
MULT_INC = 0.05

"""THRESHOLDING"""
# keyboard shortcut - t
THRESH = False
THRESH_STYLE = 0
THRESH_OPTIMIZE = 1 # 0 gives smoother results, halves fps (30->15)

"""GRADIENTS"""
# keyboard shortcut - m
GRAD_STYLE = 0

"""FFT"""
# keyboard shortcut - f
FFT = False
FFT_OFFSET = 5         # extent of central zeroing window
FFT_OFFSET_INC = 1     # amount to increment window by
# channel values for constant color (+/- grayscale ifft)
FFT_R_CH_OFFSET = 0    
FFT_G_CH_OFFSET = 0
FFT_B_CH_OFFSET = 0
FFT_CH_OFFSET_INC = 5  # amount to increment channel values by
# style indicator for none/constant/c+ifft/c-ifft
FFT_B_CH_STYLE = 0
FFT_G_CH_STYLE = 0
FFT_R_CH_STYLE = 0

"""ALIEN"""
# keyboard shortcut - a
# found in GIMP:Colors>Map>Alien Map
ALIEN = False
# style indicator for none/constant/trig
ALIEN_R_CH_STYLE = 0
ALIEN_G_CH_STYLE = 0
ALIEN_B_CH_STYLE = 0
# frequency modulation for each channel (controlled by -/=)
ALIEN_FREQ_MIN = 0
ALIEN_FREQ_MAX = 10
ALIEN_FREQ_INC = 0.05
ALIEN_R_CH_FREQ = 0.2
ALIEN_G_CH_FREQ = 0.2
ALIEN_B_CH_FREQ = 0.2
# frequency modulation for each channel (controlled by up/down)
ALIEN_PHASE_MIN = 0
ALIEN_PHASE_MAX = 2*np.pi
ALIEN_PHASE_INC = np.pi/10.0
ALIEN_R_CH_PHASE = 0
ALIEN_G_CH_PHASE = 0
ALIEN_B_CH_PHASE = 0

"""RGB_WALK"""
RGB_WALK = False
RGB_INIT = False      # flag for initializing parameters on first pass
RGB_NUM_SAMPLES = 10  # number of random samples to average for smooth walk
RGB_NOISE_INDEX = 0   # counter in noise array
RGB_CH_VEC_POS = np.zeros((3,2)) 
RGB_STEP_SIZE = 15.0   # step size in pixels that multiplies random walk step

In [ ]:
# select video source
if USE_CAM:
    cap = cv2.VideoCapture(0)
else:
    cap = cv2.VideoCapture(0)
#     cap = cv2.VideoCapture('/media/data/Dropbox/Git/vid-viz/data/tunnel_01.mp4')
#     cap = cv2.VideoCapture('/home/mattw/Videos/2016-12-18 23-15-59.mp4')
    
# get video properties
# target_fps = cap.get(cv2.CAP_PROP_FPS)
target_fps = 20.0
tot_fr_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

fr_count = 0
while(cap.isOpened()):

    time_pre = time.time()
    
    ret, frame = cap.read()
    
    if USE_CAM: #DISP_FULL_SCREEN
        # crop image to have same aspect ratio as monitor
        frame = frame[60:-60,:,:]
    
    [im_height, im_width, im_channels] = frame.shape

    """
    Get keyboard input
    """
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('s'):
        # toggle color
        DISP_BW = not DISP_BW
    elif key == ord('z'):
        BORDER_STYLE = (BORDER_STYLE + 1) % 3
    elif key == ord('R'):
        INCREMENT = True
    elif key == ord('T'):
        DECREMENT = True
    elif key == ord('t'):
        THRESH = not THRESH
    elif key == ord('y'):
        THRESH_STYLE = (THRESH_STYLE + 1) % 2
    elif key == ord('r'):
        # for thresholding
        USE_ALL_CHAN = False
        USE_R_CHAN = True
        USE_G_CHAN = False
        USE_B_CHAN = False
    elif key == ord('g'):
        # for thresholding
        USE_ALL_CHAN = False
        USE_R_CHAN = False
        USE_G_CHAN = True
        USE_B_CHAN = False
    elif key == ord('b'):
        # for thresholding
        USE_ALL_CHAN = False
        USE_R_CHAN = False
        USE_G_CHAN = False
        USE_B_CHAN = True
    elif key == ord('v'):
        USE_ALL_CHAN = True
        USE_R_CHAN = False
        USE_G_CHAN = False
        USE_B_CHAN = False
    elif key == ord('m'):
        GRAD_STYLE = (GRAD_STYLE + 1) % 2    
    elif key == ord('f'):
        FFT = not FFT
    elif key == ord('a'):
        ALIEN = not ALIEN
    elif key == ord('w'):
        RGB_WALK = not RGB_WALK
        RGB_INIT = not RGB_INIT

    elif key == ord('0'):
        KEY_PRESS_ZERO = True
    elif key == ord('1'):
        KEY_PRESS_ONE = True
    elif key == ord('2'):
        KEY_PRESS_TWO = True
    elif key == ord('3'):
        KEY_PRESS_THREE = True
    elif key == ord('-'):
        KEY_PRESS_MINUS = True
    elif key == ord('='):
        KEY_PRESS_PLUS = True
    elif key == ord('/'):
        KEY_PRESS_RESET = True
        
    """ 
    implement effects 
    """
    
#     if DISP_BW:
#         frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
#     tx = fr_count % im_height
#     ty = fr_count % im_height
#     M = np.float32([[1, 0, tx], [0, 1, ty]])
#     frame = cv2.warpAffine(frame, M, (im_width, im_height))

    
    """
    thresholding
    """
    if THRESH and not ALIEN: # hackey for now
        
        # BINARY | BINARY_INV | TRUNC | TOZERO | TOZERO_INV
        if THRESH_STYLE == 0:
            THRESH_TYPE = cv2.THRESH_BINARY
        elif THRESH_STYLE == 1:
            THRESH_TYPE = cv2.THRESH_BINARY_INV
            
        # MEAN_C | GAUSSIAN_C
        ADAPTIVE_THRESH_TYPE = cv2.ADAPTIVE_THRESH_MEAN_C
        THRESH_CEIL = 255
        THRESH_BLOCK = 21
        THRESH_C = 4
        
        if USE_ALL_CHAN:
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frame_gray = cv2.adaptiveThreshold(frame_gray, THRESH_CEIL, 
                                          ADAPTIVE_THRESH_TYPE, THRESH_TYPE, 
                                          THRESH_BLOCK, THRESH_C)
            frame_gray = cv2.medianBlur(frame_gray, 11)
            frame[:,:,0] = frame_gray
            frame[:,:,1] = frame_gray
            frame[:,:,2] = frame_gray           
        elif USE_R_CHAN:
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            if THRESH_OPTIMIZE:
                frame_gray = cv2.medianBlur(frame_gray, 11)
            # zero out blue channel, threshold red channel
            frame[:,:,0] = 0
            frame[:,:,2] = cv2.adaptiveThreshold(frame_gray, THRESH_CEIL, 
                                                 ADAPTIVE_THRESH_TYPE, THRESH_TYPE, 
                                                 THRESH_BLOCK, THRESH_C)
            if not THRESH_OPTIMIZE:
                frame = cv2.medianBlur(frame, 11)
        elif USE_G_CHAN:
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            if THRESH_OPTIMIZE:
                frame_gray = cv2.medianBlur(frame_gray, 11)
            # zero out red channel, threshold green channel
            frame[:,:,2] = 0
            frame[:,:,1] = cv2.adaptiveThreshold(frame_gray, THRESH_CEIL,
                                                 ADAPTIVE_THRESH_TYPE, THRESH_TYPE,
                                                 THRESH_BLOCK, THRESH_C)
            if not THRESH_OPTIMIZE:
                frame = cv2.medianBlur(frame, 11)
        elif USE_B_CHAN:
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            if THRESH_OPTIMIZE:
                frame_gray = cv2.medianBlur(frame_gray, 11)
            # zero out green channel, threshold red channel
            frame[:,:,1] = 0
            frame[:,:,0] = cv2.adaptiveThreshold(frame_gray, THRESH_CEIL,
                                                 ADAPTIVE_THRESH_TYPE, THRESH_TYPE,
                                                 THRESH_BLOCK, THRESH_C)
            if not THRESH_OPTIMIZE:
                frame = cv2.medianBlur(frame, 11)
    
    """
    gradients
    """
#     if GRAD_STYLE == 1:
#         grad = cv2.Sobel(frame, cv2.CV_16S, 1, 0, ksize=7)
#     elif GRAD_STYLE == 2:
#         grad = cv2.Sobel(frame, cv2.CV_16S, 0, 1, ksize=7)
#     elif GRAD_STYLE == 3:
#         grad = cv2.Scharr(frame, cv2.CV_16S, 1, 0)
#     elif GRAD_STYLE == 4:
#         grad = cv2.Scharr(frame, cv2.CV_16S, 0, 1)
#     elif GRAD_STYLE == 5:
#         grad = cv2.Laplacian(frame, cv2.CV_16S, ksize=7)
#     elif GRAD_STYLE == 6:
#         grad = cv2.Canny(frame, 100, 200)   

    if GRAD_STYLE > 0:
        grad = cv2.Canny(frame, 100, 200)  
        if USE_R_CHAN:
            frame[:,:,:2] = 0
            frame[:,:,2] = grad
        elif USE_G_CHAN:
            frame[:,:,[0,2]] = 0
            frame[:,:,1] = grad
        elif USE_B_CHAN:
            frame[:,:,1:3] = 0
            frame[:,:,0] = grad
        elif USE_ALL_CHAN:
            frame[:,:,0] = grad
            frame[:,:,1] = grad
            frame[:,:,2] = grad
        
        
    """
    FFT
    """
    if FFT:
        
        # update high-pass filter window size
        if INCREMENT:
            FFT_OFFSET += FFT_OFFSET_INC
            INCREMENT = False
        if DECREMENT:
            FFT_OFFSET -= FFT_OFFSET_INC
            if FFT_OFFSET < 0:
                FFT_OFFSET = 0
            DECREMENT = False
        if KEY_PRESS_RESET:
            FFT_R_CH_OFFSET = 0
            FFT_G_CH_OFFSET = 0
            FFT_B_CH_OFFSET = 0
            KEY_PRESS_RESET = False
            
        """ Calculate FFT """
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # pad image to optimize fft calculations
        orows, ocols = frame_gray.shape
        nrows = cv2.getOptimalDFTSize(orows)
        ncols = cv2.getOptimalDFTSize(ocols)
        frame_gray = cv2.copyMakeBorder(frame_gray, 0, nrows-orows, 0, ncols-ocols, 
                                        cv2.BORDER_CONSTANT, value=0)
        rows, cols = frame_gray.shape # update shape
        
        # take fft
        fshift = np.fft.fftshift(cv2.dft(np.float32(frame_gray), flags = cv2.DFT_COMPLEX_OUTPUT))
        
        # filter
        crow, ccol = int(rows/2), int(cols/2)
        fshift[crow-FFT_OFFSET:crow+FFT_OFFSET, ccol-FFT_OFFSET:ccol+FFT_OFFSET] = 0
        frame_back = cv2.idft(np.fft.ifftshift(fshift))
        frame_back = cv2.magnitude(frame_back[:,:,0], frame_back[:,:,1])
        frame_back = frame_back[:orows, :ocols] # update shape to original
        frame_back = (frame_back - np.min(frame_back)) / (np.max(frame_back) - np.min(frame_back)) * 255
        
        """ determine how to use resulting image """
        if USE_R_CHAN:
            
            if KEY_PRESS_MINUS:
                FFT_R_CH_OFFSET = (FFT_R_CH_OFFSET - FFT_CH_OFFSET_INC) % 255
                KEY_PRESS_MINUS = False
            if KEY_PRESS_PLUS:
                FFT_R_CH_OFFSET = (FFT_R_CH_OFFSET + FFT_CH_OFFSET_INC) % 255
                KEY_PRESS_PLUS = False
                
            if KEY_PRESS_ZERO:
                FFT_R_CH_STYLE = 0 # gray
                KEY_PRESS_ZERO = False
            if KEY_PRESS_ONE:
                FFT_R_CH_STYLE = 1 # offset
                KEY_PRESS_ONE = False
            if KEY_PRESS_TWO:
                FFT_R_CH_STYLE = 2 # offset + ifft
                KEY_PRESS_TWO = False
            if KEY_PRESS_THREE:
                FFT_R_CH_STYLE = 3 # offset - ifft
                KEY_PRESS_THREE = False
                
        elif USE_G_CHAN:
            
            if KEY_PRESS_MINUS:
                FFT_G_CH_OFFSET = (FFT_G_CH_OFFSET - FFT_CH_OFFSET_INC) % 255
                KEY_PRESS_MINUS = False
            if KEY_PRESS_PLUS:
                FFT_G_CH_OFFSET = (FFT_G_CH_OFFSET + FFT_CH_OFFSET_INC) % 255
                KEY_PRESS_PLUS = False
                
            if KEY_PRESS_ZERO:
                FFT_G_CH_STYLE = 0 # gray
                KEY_PRESS_ZERO = False
            if KEY_PRESS_ONE:
                FFT_G_CH_STYLE = 1 # offset
                KEY_PRESS_ONE = False
            if KEY_PRESS_TWO:
                FFT_G_CH_STYLE = 2 # offset + ifft
                KEY_PRESS_TWO = False
            if KEY_PRESS_THREE:
                FFT_G_CH_STYLE = 3 # offset - ifft
                KEY_PRESS_THREE = False
                
        elif USE_B_CHAN:

            if KEY_PRESS_MINUS:
                FFT_B_CH_OFFSET = (FFT_B_CH_OFFSET - FFT_CH_OFFSET_INC) % 255
                KEY_PRESS_MINUS = False
            if KEY_PRESS_PLUS:
                FFT_B_CH_OFFSET = (FFT_B_CH_OFFSET + FFT_CH_OFFSET_INC) % 255
                KEY_PRESS_PLUS = False
                
            if KEY_PRESS_ZERO:
                FFT_B_CH_STYLE = 0 # gray
                KEY_PRESS_ZERO = False
            if KEY_PRESS_ONE:
                FFT_B_CH_STYLE = 1 # offset
                KEY_PRESS_ONE = False
            if KEY_PRESS_TWO:
                FFT_B_CH_STYLE = 2 # offset + ifft
                KEY_PRESS_TWO = False
            if KEY_PRESS_THREE:
                FFT_B_CH_STYLE = 3 # offset - ifft
                KEY_PRESS_THREE = False
        
        """ update frame """
        frame = np.zeros(shape=frame.shape, dtype='uint8')
        
        if FFT_R_CH_STYLE == 0:
            frame[:,:,2] = frame_gray
        elif FFT_R_CH_STYLE == 1:
            frame[:,:,2] = FFT_R_CH_OFFSET
        elif FFT_R_CH_STYLE == 2:
            frame[:,:,2] = FFT_R_CH_OFFSET + frame_back.astype(np.uint8)
        elif FFT_R_CH_STYLE == 3:
            frame[:,:,2] = FFT_R_CH_OFFSET - frame_back.astype(np.uint8)
            
        if FFT_G_CH_STYLE == 0:
            frame[:,:,1] = frame_gray
        elif FFT_G_CH_STYLE == 1:
            frame[:,:,1] = FFT_G_CH_OFFSET
        elif FFT_G_CH_STYLE == 2:
            frame[:,:,1] = FFT_G_CH_OFFSET + frame_back.astype(np.uint8)
        elif FFT_G_CH_STYLE == 3:
            frame[:,:,1] = FFT_G_CH_OFFSET - frame_back.astype(np.uint8)
            
        if FFT_B_CH_STYLE == 0:
            frame[:,:,0] = frame_gray
        elif FFT_B_CH_STYLE == 1:
            frame[:,:,0] = FFT_B_CH_OFFSET
        elif FFT_B_CH_STYLE == 2:
            frame[:,:,0] = FFT_B_CH_OFFSET + frame_back.astype(np.uint8)
        elif FFT_B_CH_STYLE == 3:
            frame[:,:,0] = FFT_B_CH_OFFSET - frame_back.astype(np.uint8)
            
        frame = cv2.medianBlur(frame, 9)
    
    """
    ALIEN
    """
    if ALIEN:
        
        # start with smoothed image
        frame = cv2.GaussianBlur(frame, (11,11), 0)
        
        # update params
        if KEY_PRESS_RESET:
            ALIEN_R_CH_FREQ = 0.2
            ALIEN_G_CH_FREQ = 0.2
            ALIEN_B_CH_FREQ = 0.2
            ALIEN_R_CH_PHASE = 0
            ALIEN_G_CH_PHASE = 0
            ALIEN_B_CH_PHASE = 0
            KEY_PRESS_RESET = False
        
        """determine how to update params"""
        if USE_R_CHAN:
            
            # update freq
            if KEY_PRESS_MINUS:
                ALIEN_R_CH_FREQ -= ALIEN_FREQ_INC
                KEY_PRESS_MINUS = False
            if KEY_PRESS_PLUS:
                ALIEN_R_CH_FREQ += ALIEN_FREQ_INC
                KEY_PRESS_PLUS = False
            ALIEN_R_CH_FREQ = np.clip(ALIEN_R_CH_FREQ, ALIEN_FREQ_MIN, ALIEN_FREQ_MAX)
                
            # update phase
            if DECREMENT:
                ALIEN_R_CH_PHASE = (ALIEN_R_CH_PHASE - ALIEN_PHASE_INC) % ALIEN_PHASE_MAX
                DECREMENT = False
            if INCREMENT:
                ALIEN_R_CH_PHASE = (ALIEN_R_CH_PHASE + ALIEN_PHASE_INC) % ALIEN_PHASE_MAX
                INCREMENT = False
                    
            if KEY_PRESS_ZERO:
                ALIEN_R_CH_STYLE = 0 # normal
                KEY_PRESS_ZERO = False
            if KEY_PRESS_ONE:
                ALIEN_R_CH_STYLE = 1 # offset = 0
                KEY_PRESS_ONE = False
            if KEY_PRESS_TWO:
                ALIEN_R_CH_STYLE = 2 # offset = 255
                KEY_PRESS_TWO = False
            if KEY_PRESS_THREE:
                ALIEN_R_CH_STYLE = 3 # trig
                KEY_PRESS_THREE = False
                
        elif USE_G_CHAN:
            
            # update freq
            if KEY_PRESS_MINUS:
                ALIEN_G_CH_FREQ -= ALIEN_FREQ_INC
                KEY_PRESS_MINUS = False
            if KEY_PRESS_PLUS:
                ALIEN_G_CH_FREQ += ALIEN_FREQ_INC
                KEY_PRESS_PLUS = False
            ALIEN_G_CH_FREQ = np.clip(ALIEN_G_CH_FREQ, ALIEN_FREQ_MIN, ALIEN_FREQ_MAX)
                
            # update phase
            if DECREMENT:
                ALIEN_G_CH_PHASE = (ALIEN_G_CH_PHASE - ALIEN_PHASE_INC) % ALIEN_PHASE_MAX
                DECREMENT = False
            if INCREMENT:
                ALIEN_G_CH_PHASE = (ALIEN_G_CH_PHASE + ALIEN_PHASE_INC) % ALIEN_PHASE_MAX
                INCREMENT = False
                
            if KEY_PRESS_ZERO:
                ALIEN_G_CH_STYLE = 0 # normal
                KEY_PRESS_ZERO = False
            if KEY_PRESS_ONE:
                ALIEN_G_CH_STYLE = 1 # offset = 0
                KEY_PRESS_ONE = False
            if KEY_PRESS_TWO:
                ALIEN_G_CH_STYLE = 2 # offset = 255
                KEY_PRESS_TWO = False
            if KEY_PRESS_THREE:
                ALIEN_G_CH_STYLE = 3 # trig
                KEY_PRESS_THREE = False
                
        elif USE_B_CHAN:

            # update freq
            if KEY_PRESS_MINUS:
                ALIEN_B_CH_FREQ -= ALIEN_FREQ_INC
                KEY_PRESS_MINUS = False
            if KEY_PRESS_PLUS:
                ALIEN_B_CH_FREQ += ALIEN_FREQ_INC
                KEY_PRESS_PLUS = False
            ALIEN_B_CH_FREQ = np.clip(ALIEN_B_CH_FREQ, ALIEN_FREQ_MIN, ALIEN_FREQ_MAX)
            
            # update phase
            if DECREMENT:
                ALIEN_B_CH_PHASE = (ALIEN_B_CH_PHASE - ALIEN_PHASE_INC) % ALIEN_PHASE_MAX
                DECREMENT = False
            if INCREMENT:
                ALIEN_B_CH_PHASE = (ALIEN_B_CH_PHASE + ALIEN_PHASE_INC) % ALIEN_PHASE_MAX
                INCREMENT = False
                
            if KEY_PRESS_ZERO:
                ALIEN_B_CH_STYLE = 0 # normal
                KEY_PRESS_ZERO = False
            if KEY_PRESS_ONE:
                ALIEN_B_CH_STYLE = 1 # offset = 0
                KEY_PRESS_ONE = False
            if KEY_PRESS_TWO:
                ALIEN_B_CH_STYLE = 2 # offset = 255
                KEY_PRESS_TWO = False
            if KEY_PRESS_THREE:
                ALIEN_B_CH_STYLE = 3 # trig
                KEY_PRESS_THREE = False
        
        """ update frame """
        frame_orig = frame
        frame_orig.astype('float16')
        frame.astype('float16')
        
        NUM_LEVELS = 10.0
        if ALIEN_R_CH_STYLE == 0:
            frame[:,:,2] = frame_orig[:,:,2]
        elif ALIEN_R_CH_STYLE == 1:
            frame[:,:,2] = 0
        elif ALIEN_R_CH_STYLE == 2:
            frame[:,:,2] = 255
        elif ALIEN_R_CH_STYLE == 3:
            frame[:,:,2] = 128 + 127*np.cos(frame[:,:,2]*ALIEN_R_CH_FREQ + ALIEN_R_CH_PHASE)
#             if THRESH:
#                 frame[:,:,2] = np.floor(frame[:,:,2]/255.0*NUM_LEVELS)/NUM_LEVELS*255.0
            
        if ALIEN_G_CH_STYLE == 0:
            frame[:,:,1] = frame_orig[:,:,1]
        elif ALIEN_G_CH_STYLE == 1:
            frame[:,:,1] = 0
        elif ALIEN_G_CH_STYLE == 2:
            frame[:,:,1] = 255
        elif ALIEN_G_CH_STYLE == 3:
            frame[:,:,1] = 128 + 127*np.cos(frame[:,:,1]*ALIEN_G_CH_FREQ + ALIEN_G_CH_PHASE)
#             if THRESH:
#                 frame[:,:,1] = np.floor(frame[:,:,1]/255.0*NUM_LEVELS)/NUM_LEVELS*255.0
                
        if ALIEN_B_CH_STYLE == 0:
            frame[:,:,0] = frame_orig[:,:,0]
        elif ALIEN_B_CH_STYLE == 1:
            frame[:,:,0] = 0
        elif ALIEN_B_CH_STYLE == 2:
            frame[:,:,0] = 255
        elif ALIEN_B_CH_STYLE == 3:
            frame[:,:,0] = 128 + 127*np.cos(frame[:,:,0]*ALIEN_B_CH_FREQ + ALIEN_B_CH_PHASE)
#         if THRESH:
#                 frame[:,:,0] = np.floor(frame[:,:,0]/255.0*NUM_LEVELS)/NUM_LEVELS*255.0
        
                    
        frame.astype('uint8')

        if THRESH:
            frame = cv2.medianBlur(frame, 9)
        else:
            frame = cv2.GaussianBlur(frame, (9,9), 0)
        
        
        # threshold to clean up effect
        if THRESH:
            if THRESH_STYLE == 0:
                THRESH_TYPE = cv2.THRESH_BINARY
            elif THRESH_STYLE == 1:
                THRESH_TYPE = cv2.THRESH_BINARY_INV

            # MEAN_C | GAUSSIAN_C
            ADAPTIVE_THRESH_TYPE = cv2.ADAPTIVE_THRESH_MEAN_C
            THRESH_CEIL = 255
            THRESH_BLOCK = 15
            THRESH_C = 5

            if ALIEN_R_CH_STYLE == 3:
                frame[:,:,2] = cv2.adaptiveThreshold(frame[:,:,2], THRESH_CEIL, 
                                                     ADAPTIVE_THRESH_TYPE, THRESH_TYPE, 
                                                     THRESH_BLOCK, THRESH_C)
            if ALIEN_G_CH_STYLE == 3:
                frame[:,:,1] = cv2.adaptiveThreshold(frame[:,:,1], THRESH_CEIL, 
                                                     ADAPTIVE_THRESH_TYPE, THRESH_TYPE, 
                                                     THRESH_BLOCK, THRESH_C)
            if ALIEN_B_CH_STYLE == 3:
                frame[:,:,0] = cv2.adaptiveThreshold(frame[:,:,0], THRESH_CEIL, 
                                                     ADAPTIVE_THRESH_TYPE, THRESH_TYPE, 
                                                     THRESH_BLOCK, THRESH_C)
            frame = cv2.medianBlur(frame, 11)
        else:               
            frame = cv2.GaussianBlur(frame, (9,9), 0)
        
        
    """
    RGB WALK
    """
    if RGB_WALK:
        
        if RGB_INIT:
            noise_samples = np.random.randn(RGB_NUM_SAMPLES,6) # x,y for RGB channels
            RGB_INIT = False
            
        # update params
        if KEY_PRESS_RESET:
            RGB_CH_VEC_POS = np.zeros((3,2), dtype='float64')
            KEY_PRESS_RESET = False
            
        # process image
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_gray = cv2.adaptiveThreshold(frame_gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, 
                                           cv2.THRESH_BINARY_INV, 15, 5)
        frame_gray = cv2.medianBlur(frame_gray, 7)
        
        # update noise values
        noise_samples[RGB_NOISE_INDEX,:] = np.random.randn(1,6)
        RGB_NOISE_INDEX = (RGB_NOISE_INDEX + 1) % RGB_NUM_SAMPLES
        RGB_CH_VEC_POS += np.reshape(RGB_STEP_SIZE*np.mean(noise_samples, axis=0), (3,2))
        
        # translate image
        frame[:,:,0] = cv2.warpAffine(frame_gray, 
                                      np.float32([[1, 0, RGB_CH_VEC_POS[0,0]], 
                                                  [0, 1, RGB_CH_VEC_POS[0,1]]]), 
                                      (im_width,im_height))
        frame[:,:,1] = cv2.warpAffine(frame_gray, 
                                      np.float32([[1, 0, RGB_CH_VEC_POS[1,0]], 
                                                  [0, 1, RGB_CH_VEC_POS[1,1]]]), 
                                      (im_width,im_height))
        frame[:,:,2] = cv2.warpAffine(frame_gray, 
                                      np.float32([[1, 0, RGB_CH_VEC_POS[2,0]], 
                                                  [0, 1, RGB_CH_VEC_POS[2,1]]]), 
                                      (im_width,im_height))

    
    """
    Apply borders last to save processing time
    """
    if BORDER_STYLE > 0:
    
        if INCREMENT:
            HEIGHT_MULT += MULT_INC
            WIDTH_MULT += MULT_INC
            INCREMENT = False
        if DECREMENT:
            HEIGHT_MULT -= MULT_INC
            if HEIGHT_MULT < 0:
                HEIGHT_MULT = 0
            WIDTH_MULT -= MULT_INC
            if WIDTH_MULT < 0:
                WIDTH_MULT = 0
            DECREMENT = False
        
        if BORDER_STYLE == 1:

            # top, bottom, left, right
            frame = cv2.copyMakeBorder(frame, int(im_height*HEIGHT_MULT), int(im_height*HEIGHT_MULT),
                                              int(im_width*WIDTH_MULT), int(im_width*WIDTH_MULT),
                                              cv2.BORDER_WRAP)
        elif BORDER_STYLE == 2:
            # top, bottom, left, right
            frame = cv2.copyMakeBorder(frame, int(im_height*2*HEIGHT_MULT), 0,
                                              int(im_width*2*WIDTH_MULT), 0,
                                              cv2.BORDER_REFLECT)
        
    """
    diplay image
    """
    if DISP_FULL_SCREEN:
        cv2.namedWindow('frame', cv2.WND_PROP_FULLSCREEN)
        cv2.setWindowProperty('frame', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
        cv2.imshow('frame', frame)
    else:
        cv2.imshow('frame', frame)
        if fr_count == 0:
            cv2.moveWindow('frame', 0, 0)
    
    """
    clean up
    """
    # reset video
    fr_count += 1
    if fr_count == tot_fr_count and LOOP:
        # reset frame postion to zero
        cap.set(cv2.CAP_PROP_POS_FRAMES,1)
        fr_count = 1
    
    # calculate, limit and output fps
    time_post = time.time()
    time_tot = time_post - time_pre
    if time_tot < 1/target_fps:
        time.sleep(1/target_fps - time_tot)
    time_post = time.time()
    time_tot = time_post - time_pre
    fps = 1.0 / time_tot
    print('\r%03i fps' % fps, end='')
    
cap.release()
cv2.destroyWindow('frame')